In [1]:
import $ivy.`com.typesafe.akka::akka-actor:2.6.10`
import akka.actor.{Actor, ActorRef, Props}
import akka.actor.{ActorSystem, PoisonPill, Props}

import $ivy.$
import akka.actor.{Actor, ActorRef, Props}
import akka.actor.{ActorSystem, PoisonPill, Props}

In [2]:
case class Greet(name: String)
case class Praise(name: String)
case class Celebrate(name: String, age: Int)

defined class Greet
defined class Praise
defined class Celebrate

In [3]:
class Talker extends Actor {
    def receive = {
        case Greet(name)          => println(s"Hello $name")
        case Praise(name)         => println(s"$name, you're amazing")
        case Celebrate(name, age) => println(s"Here's to another $age years, $name")
    }
}

defined class Talker

In [4]:
object HelloActors {
    val system = ActorSystem("HelloActors")
    val talker = system.actorOf(Props(new Talker), "talker")
    talker ! Greet("Huey")
    talker ! Praise("Dewey")
    talker ! Celebrate("Louie", 16)
    Thread.sleep(2000)
    system.terminate()
}

defined object HelloActors

In [5]:
HelloActors

Hello Huey
Dewey, you're amazing
Here's to another 16 years, Louie


res5: HelloActors.type = ammonite.$sess.cmd4$Helper$HelloActors$@4942b3b1